In [ ]:
!pip install transformers
!pip install torch
!pip install newspaper3k

In [ ]:
import nltk
nltk.download('punkt')
from textblob import TextBlob
from newspaper import Article

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [3]:
from dataclasses import dataclass

In [4]:
@dataclass
class Mood:
    emoji: str
    sentiment: float

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")


In [6]:
def sentiment_analysis(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    labels = ['negative', 'neutral', 'positive']
    result = {labels[i]: scores[i] for i in range(len(labels))}
    return result


In [15]:
def get_mood(input_text: str, *, threshold: float) -> Mood:
    sentiment: float = TextBlob(input_text).sentiment.polarity


    friendly_treshold: float = threshold
    hostile_treshold: float = -threshold

    if sentiment > friendly_treshold:
        return Mood("😊", sentiment)
    elif sentiment < hostile_treshold:
        return Mood("😠", sentiment)
    else:
        return Mood("😐", sentiment)



url = "https://www.bbc.com/news/articles/cv22ydr7v3ko"
article = Article(url)
article.download()
article.parse()
article.nlp()

text = article.summary
print(text)

blob = TextBlob(text)
sentiment = blob.sentiment.polarity # 0 to 1
print(sentiment)

mood = get_mood(text, threshold=0.1)
print(mood)

Four men jailed over 'senseless' murder of cyclist24 hours ago ShareFamily Photo Kyron Lee was stabbed to death in October 2022Four men who stabbed a cyclist to death in a "senseless" murder after knocking him down in a stolen car have been jailed.
Kyron Lee, 21, was pronounced dead at the scene in Cippenham, Slough, in October 2022.
Mohammed Elgamri, 20, of Carter Close, Windsor, was sentenced to life with a minimum of 25 years after being found guilty of murder.
Yaqhub Mussa, 23, of Daylesford Grove, Slough, was sentenced to life with a minimum of 23 years and nine months after pleading guilty to murder.
Fras Seedahmed, 19, of Surrey Avenue, Slough, was sentenced to life with a minimum of 23 years after being found guilty of murder.
-0.3711111111111111
Mood(emoji='😠', sentiment=-0.3711111111111111)
